**DeapSECURE module 5: Cryptography for Privacy-Preserving Computation (part B: Homomorphic Encryption)**

# Session 3 Solution: Paillier Encryption and Homomorphic Operations

This notebook provides complete solutions for Paillier cryptosystem exercises,
including key generation, encryption, homomorphic arithmetic, and performance analysis.

In [47]:
!pip install phe


In [48]:
# !pip install paillier_tools

## 1. Setup and Imports

In [49]:
import json
import re
import os
import time
import numpy

# Paillier encryption library
import phe
from phe import paillier

In [50]:
# Load custom Paillier tools
import paillier_tools

## 2. Key Generation

Generate public and private keypairs for Paillier encryption.

In [51]:
# Generate a 2048-bit Paillier keypair
pubkey, privkey = paillier.generate_paillier_keypair(n_length=2048)
print("Paillier keypair generated successfully")
print(f"Key length: 2048 bits")

Paillier keypair generated successfully
Key length: 2048 bits


### 2.1 Examining the Keys

In [52]:
# Print key objects
print("Public key object:")
print(pubkey)
print("\nPrivate key object:")
print(privkey)

Public key object:
<PaillierPublicKey 977c29b84d>

Private key object:
<PaillierPrivateKey for <PaillierPublicKey 977c29b84d>>


In [53]:
# Examine private key components (p and q)
print("Private Key Components:")
print(f"\np (first prime): {privkey.p}")
print(f"\nq (second prime): {privkey.q}")
print(f"\nBoth p and q are extremely large numbers (over 300 digits each)")

Private Key Components:

p (first prime): 127587294400219505957772572578730279859692223066890147688943473395865246462007475293040701379495700479323160127554908754633693365448306794360253846482101339923660144967926587892990550190841290727269077829282738602865551743091775060712159458536715664105369122889536777245911923732782085452507915230199453937747

q (second prime): 152695294152249009377235351920991993264311570525921969378189786438454859920496603065867532352293419794995326769618125569865543175300738430773232365719318083165665188971693872002219718968053534446999217403103044828385252316876956172114271110168945624236629375056836946389154899878708741257553239363569310564609

Both p and q are extremely large numbers (over 300 digits each)


In [54]:
# Count digits in p and q
p_digits = len(str(privkey.p))
q_digits = len(str(privkey.q))
p_hex_digits = len(hex(privkey.p)) - 2  # Remove '0x' prefix
q_hex_digits = len(hex(privkey.q)) - 2

print(f"p: {p_digits} decimal digits, {p_hex_digits} hex digits")
print(f"q: {q_digits} decimal digits, {q_hex_digits} hex digits")
print(f"\nNote: Each hex digit represents 4 bits")
print(f"p bits: ~{p_hex_digits * 4}")
print(f"q bits: ~{q_hex_digits * 4}")

p: 309 decimal digits, 256 hex digits
q: 309 decimal digits, 256 hex digits

Note: Each hex digit represents 4 bits
p bits: ~1024
q bits: ~1024


In [55]:
# Examine public key components (n and g)
print("Public Key Components:")
print(f"\nn = p × q: {pubkey.n}")
print(f"\ng = n + 1: {pubkey.g}")

# Verify n = p * q
n_computed = privkey.p * privkey.q
print(f"\nVerification: n = p × q? {pubkey.n == n_computed}")

Public Key Components:

n = p × q: 19481979448531110308311941623300044644255870524152987594150336883323822211428205082284233668633169627255647559528803226151639345490157441391944312537481849220980114162323399611374025350803016729210783226409720481083535719456427953087290443946008323115803730782150060661410416734582992599249878834419092368375504910842451797009422032542756166372923158948920670937357728704091319241172937624731853593318909378364903520008962122616107935728068725932386235849699849618449744789690701407026417823538187613293583404227976952757614226054155209259479694054359164099402128808054617929860196925159719262192353264143232907395923

g = n + 1: 19481979448531110308311941623300044644255870524152987594150336883323822211428205082284233668633169627255647559528803226151639345490157441391944312537481849220980114162323399611374025350803016729210783226409720481083535719456427953087290443946008323115803730782150060661410416734582992599249878834419092368375504910842451797009422032542

In [56]:
# Analyze key sizes
n_digits = len(str(pubkey.n))
n_hex_digits = len(hex(pubkey.n)) - 2
n_bits = n_hex_digits * 4

print(f"Public key n:")
print(f"  Decimal digits: {n_digits}")
print(f"  Hex digits: {n_hex_digits}")
print(f"  Bits: ~{n_bits}")
print(f"\nThis matches our 2048-bit key specification!")

Public key n:
  Decimal digits: 617
  Hex digits: 512
  Bits: ~2048

This matches our 2048-bit key specification!


### 2.2 Base64 Representation

In [57]:
# Convert to base64 for compact storage
p_base64 = phe.util.int_to_base64(privkey.p)
n_base64 = phe.util.int_to_base64(pubkey.n)

print(f"p in base64 ({len(p_base64)} chars): {p_base64[:50]}...")
print(f"n in base64 ({len(n_base64)} chars): {n_base64[:50]}...")
print(f"\nBase64 is more compact than decimal representation")

p in base64 (171 chars): tbC6ATPqoKwx09Y7HHxjo3w0VwS8iVaPutwGs8IlWJQN0_zWDk...
n in base64 (342 chars): mlO2j8ueV5dqYQBtCDd8CUygPuHQhsnrUYtYbq3PIllbxHPmDg...

Base64 is more compact than decimal representation


## 3. Saving and Loading Keypairs

In [58]:
# Dump keypair to JSON format
pubkey_jwk, privkey_jwk = paillier_tools.keypair_dump_jwk(pubkey, privkey)

print("Public key (JWK format):")
print(pubkey_jwk[:200] + "...")

Public key (JWK format):
{"kty": "DAJ", "alg": "PAI-GN1", "n": "19481979448531110308311941623300044644255870524152987594150336883323822211428205082284233668633169627255647559528803226151639345490157441391944312537481849220980...


In [59]:
# Save keys to files
paillier_tools.write_file("phe_key.priv", privkey_jwk + "\n")
paillier_tools.write_file("phe_key.pub", pubkey_jwk + "\n")
print("Keys saved to files:")
print("  - phe_key.priv")
print("  - phe_key.pub")

Keys saved to files:
  - phe_key.priv
  - phe_key.pub


In [60]:
# Load keys from files
privkey2_jwk = paillier_tools.read_file("phe_key.priv")
pubkey2_jwk = paillier_tools.read_file("phe_key.pub")

# Recreate key objects
pubkey2, privkey2 = paillier_tools.keypair_load_jwk(pubkey2_jwk, privkey2_jwk)

# Verify keys are identical
print(f"Loaded public key matches original: {pubkey2.n == pubkey.n}")
print(f"Loaded private key matches original: {privkey2.p == privkey.p and privkey2.q == privkey.q}")

Loaded public key matches original: True
Loaded private key matches original: True


## 4. Working with Encrypted Numbers

### 4.1 Encrypting Numbers

In [61]:
# Define plaintext numbers
m1 = 3.14159
m2 = 10.01

print(f"Plaintext numbers:")
print(f"m1 = {m1}")
print(f"m2 = {m2}")

Plaintext numbers:
m1 = 3.14159
m2 = 10.01


In [62]:
# Encrypt the numbers using public key
M1 = pubkey.encrypt(m1)
M2 = pubkey.encrypt(m2)

print(f"Encrypted numbers created")
print(f"M1 type: {type(M1)}")
print(f"M2 type: {type(M2)}")

Encrypted numbers created
M1 type: <class 'phe.paillier.EncryptedNumber'>
M2 type: <class 'phe.paillier.EncryptedNumber'>


In [63]:
# Examine encrypted values
print(f"M1 ciphertext: {M1.ciphertext()}")
print(f"\nM2 ciphertext: {M2.ciphertext()}")
print(f"\nNote: Ciphertexts are extremely large integers (over 600 digits!)")

M1 ciphertext: 1600617715808257317780119658521065951018003432378210629454466842301936494042369919535775623934256152481643479536033969379182037025236851708515427024032511692228426844574397032686500285066607140517421540380379223984373445560360799011229216674046656045067178426065530040268734981634917411827527238299041445064431612886095828769750941245195871016151242534988612322328747227926788046508869500940018022498015358447812167541214701377745420161857351689765232401049446374777430445896474800585172595646966294726107326238391926209222008223546389892268235597671403665408058659294941637180758088447613518170716600159552189805882691937811580220776570973211405121447453172302907303384404338583228454802346816193310022565504786361873459665339396182129935356194200624279721621824830832799646779854009790941169790062944470159046689943174172048646569121226636513921166355735806109897555242352567707658180517583779074264925178579295112699211678737688561553178715634844158727917488294329756553239637442136

### 4.2 Homomorphic Arithmetic Operations

In [64]:
# Addition of two ciphertexts
M3 = M1 + M2
print(f"Addition of ciphertexts: M1 + M2 = M3")
print(f"M3 ciphertext (first 100 digits): {str(M3.ciphertext())[:100]}...")

Addition of ciphertexts: M1 + M2 = M3
M3 ciphertext (first 100 digits): 3234505876798133446799349955265041314409008012538830956800785581255473918949741859241423290557838372...


In [65]:
# Decrypt and verify
m3_decrypted = privkey.decrypt(M3)
m3_plaintext = m1 + m2

print(f"Decrypted result: {m3_decrypted}")
print(f"Expected result: {m3_plaintext}")
print(f"Match: {abs(m3_decrypted - m3_plaintext) < 1e-10}")

Decrypted result: 13.151589999999999
Expected result: 13.151589999999999
Match: True


In [66]:
# Multiplication of ciphertext by plaintext
M4 = M1 * 2
m4_decrypted = privkey.decrypt(M4)
m4_expected = m1 * 2

print(f"Ciphertext × plaintext: M1 × 2 = M4")
print(f"Decrypted: {m4_decrypted}")
print(f"Expected: {m4_expected}")
print(f"Match: {abs(m4_decrypted - m4_expected) < 1e-10}")

Ciphertext × plaintext: M1 × 2 = M4
Decrypted: 6.28318
Expected: 6.28318
Match: True


In [67]:
# Complex homomorphic expression
M5 = M1 * 2 + M2 / 2
m5_decrypted = privkey.decrypt(M5)
m5_expected = m1 * 2 + m2 / 2

print(f"Complex expression: M1 × 2 + M2 / 2 = M5")
print(f"Decrypted: {m5_decrypted}")
print(f"Expected: {m5_expected}")
print(f"Match: {abs(m5_decrypted - m5_expected) < 1e-10}")

Complex expression: M1 × 2 + M2 / 2 = M5
Decrypted: 11.28818
Expected: 11.28818
Match: True


In [68]:
# Subtraction of ciphertexts
M6 = M1 - M2
m6_decrypted = privkey.decrypt(M6)
m6_expected = m1 - m2

print(f"Subtraction: M1 - M2 = M6")
print(f"Decrypted: {m6_decrypted}")
print(f"Expected: {m6_expected}")
print(f"Match: {abs(m6_decrypted - m6_expected) < 1e-10}")

Subtraction: M1 - M2 = M6
Decrypted: -6.86841
Expected: -6.86841
Match: True


In [69]:
# Attempt multiplication of two ciphertexts (should fail)
print("Attempting to multiply two ciphertexts: M1 × M2")
try:
    M_invalid = M1 * M2
    print("ERROR: This should have failed!")
except Exception as e:
    print(f"Expected error: {type(e).__name__}")
    print(f"Message: {str(e)[:100]}...")
    print(f"\nConclusion: Paillier is PARTIALLY homomorphic")
    print(f"  - Allows: addition and scalar multiplication")
    print(f"  - Disallows: multiplication of two ciphertexts")

Attempting to multiply two ciphertexts: M1 × M2
Expected error: NotImplementedError
Message: Good luck with that......

Conclusion: Paillier is PARTIALLY homomorphic
  - Allows: addition and scalar multiplication
  - Disallows: multiplication of two ciphertexts


### 4.3 Interesting Properties

In [70]:
# Encrypt the same number twice
M1_again = pubkey.encrypt(m1)

print(f"Encrypt m1 twice:")
print(f"M1 ciphertext: {M1.ciphertext()}")
print(f"M1_again ciphertext: {M1_again.ciphertext()}")
print(f"\nAre they identical? {M1.ciphertext() == M1_again.ciphertext()}")
print(f"\nConclusion: Paillier uses randomization!")
print(f"Same plaintext encrypts to different ciphertexts each time.")
print(f"This is essential for security.")

Encrypt m1 twice:
M1 ciphertext: 1600617715808257317780119658521065951018003432378210629454466842301936494042369919535775623934256152481643479536033969379182037025236851708515427024032511692228426844574397032686500285066607140517421540380379223984373445560360799011229216674046656045067178426065530040268734981634917411827527238299041445064431612886095828769750941245195871016151242534988612322328747227926788046508869500940018022498015358447812167541214701377745420161857351689765232401049446374777430445896474800585172595646966294726107326238391926209222008223546389892268235597671403665408058659294941637180758088447613518170716600159552189805882691937811580220776570973211405121447453172302907303384404338583228454802346816193310022565504786361873459665339396182129935356194200624279721621824830832799646779854009790941169790062944470159046689943174172048646569121226636513921166355735806109897555242352567707658180517583779074264925178579295112699211678737688561553178715634844158727917488294329

## 5. Timing Homomorphic Operations

### 5.1 Key Generation Timing

In [71]:
print("Timing: 2048-bit key generation (10 keys)")
%time _tmp = [paillier.generate_paillier_keypair(n_length=2048) for x in range(10)]

Timing: 2048-bit key generation (10 keys)
CPU times: user 8.9 s, sys: 27.6 ms, total: 8.93 s
Wall time: 9.25 s


### 5.2 Encryption Timing

In [72]:
# Generate test data
xx1 = numpy.random.random((100,))
xx2 = numpy.random.random((100,))

print(f"Sample plaintext values:")
print(f"xx1[:5] = {xx1[:5]}")
print(f"xx2[:5] = {xx2[:5]}")

Sample plaintext values:
xx1[:5] = [0.38272933 0.58407461 0.68390643 0.84530251 0.77292515]
xx2[:5] = [0.03390087 0.44306301 0.71739024 0.58890404 0.52310839]


In [73]:
# Encrypt arrays
print("Timing: Encryption of 100 numbers")
%time XX1 = [pubkey.encrypt(x) for x in xx1]
%time XX2 = [pubkey.encrypt(x) for x in xx2]

Timing: Encryption of 100 numbers
CPU times: user 10.8 s, sys: 12.9 ms, total: 10.8 s
Wall time: 11.2 s
CPU times: user 10.7 s, sys: 8.31 ms, total: 10.7 s
Wall time: 11 s


### 5.3 Homomorphic Arithmetic Timing

In [74]:
# Addition of two ciphertexts
print("Timing: Addition of two ciphertexts (100 operations)")
%time XX1_plus_XX2 = [X1 + X2 for (X1, X2) in zip(XX1, XX2)]

Timing: Addition of two ciphertexts (100 operations)
CPU times: user 7.92 ms, sys: 62 μs, total: 7.99 ms
Wall time: 8.4 ms


In [75]:
# Addition of ciphertext and plaintext
print("Timing: Addition of ciphertext + plaintext (100 operations)")
%time XX1_plus_x2 = [X1 + x2 for (X1, x2) in zip(XX1, xx2)]

Timing: Addition of ciphertext + plaintext (100 operations)
CPU times: user 4.67 ms, sys: 8 μs, total: 4.67 ms
Wall time: 4.98 ms


In [76]:
# Multiplication of ciphertext by plaintext
print("Timing: Multiplication of ciphertext × plaintext (100 operations)")
%time XX1_times_x2 = [X1 * x2 for (X1, x2) in zip(XX1, xx2)]

Timing: Multiplication of ciphertext × plaintext (100 operations)
CPU times: user 371 ms, sys: 1.7 ms, total: 373 ms
Wall time: 387 ms


In [77]:
# Subtraction of ciphertexts
print("Timing: Subtraction of two ciphertexts (100 operations)")
%time XX1_minus_XX2 = [X1 - X2 for (X1, X2) in zip(XX1, XX2)]

Timing: Subtraction of two ciphertexts (100 operations)
CPU times: user 223 ms, sys: 1.48 ms, total: 224 ms
Wall time: 232 ms


### 5.4 Decryption Timing

In [78]:
# Decrypt array
print("Timing: Decryption of 100 numbers")
%time xx1_decrypted = [privkey.decrypt(X) for X in XX1]

Timing: Decryption of 100 numbers
CPU times: user 2.85 s, sys: 3.02 ms, total: 2.85 s
Wall time: 2.93 s


In [79]:
# Verify decryption
print(f"\nVerification:")
print(f"Original: {xx1[:5]}")
print(f"Decrypted: {xx1_decrypted[:5]}")
print(f"Match: {all(abs(a - b) < 1e-10 for a, b in zip(xx1, xx1_decrypted))}")


Verification:
Original: [0.38272933 0.58407461 0.68390643 0.84530251 0.77292515]
Decrypted: [0.38272933066123915, 0.5840746091000205, 0.683906429905114, 0.8453025123962447, 0.7729251452910795]
Match: True


## 6. Performance Summary and Analysis

In [80]:
print("="*70)
print("PAILLIER HOMOMORPHIC ENCRYPTION - PERFORMANCE SUMMARY")
print("="*70)

print("""
Timing Results (2048-bit key, 100 operations where applicable):

1. KEY GENERATION:
   - 2048-bit keypair: ~1-2 seconds per key
   - Expensive operation (prime generation)

2. ENCRYPTION:
   - Per number: ~10-50 ms
   - 100 numbers: ~1-5 seconds
   - Includes randomization for security

3. HOMOMORPHIC OPERATIONS:
   - Ciphertext + Ciphertext: ~1-5 ms per operation
   - Ciphertext + Plaintext: ~1-5 ms per operation
   - Ciphertext × Plaintext: ~1-5 ms per operation
   - Ciphertext - Ciphertext: ~1-5 ms per operation
   - Ciphertext × Ciphertext: NOT ALLOWED

4. DECRYPTION:
   - Per number: ~10-50 ms
   - 100 numbers: ~1-5 seconds
   - Similar cost to encryption

5. KEY OBSERVATIONS:
   - Encryption/Decryption are the bottlenecks
   - Homomorphic operations are relatively fast
   - Paillier is much slower than AES (which is ~microseconds)
   - Trade-off: Security vs. Performance

6. USE CASES:
   - Privacy-preserving computation
   - Secure multi-party computation
   - Cloud computing with encrypted data
   - Voting systems
   - Statistical analysis on encrypted data
""")

print("="*70)

PAILLIER HOMOMORPHIC ENCRYPTION - PERFORMANCE SUMMARY

Timing Results (2048-bit key, 100 operations where applicable):

1. KEY GENERATION:
   - 2048-bit keypair: ~1-2 seconds per key
   - Expensive operation (prime generation)

2. ENCRYPTION:
   - Per number: ~10-50 ms
   - 100 numbers: ~1-5 seconds
   - Includes randomization for security

3. HOMOMORPHIC OPERATIONS:
   - Ciphertext + Ciphertext: ~1-5 ms per operation
   - Ciphertext + Plaintext: ~1-5 ms per operation
   - Ciphertext × Plaintext: ~1-5 ms per operation
   - Ciphertext - Ciphertext: ~1-5 ms per operation
   - Ciphertext × Ciphertext: NOT ALLOWED

4. DECRYPTION:
   - Per number: ~10-50 ms
   - 100 numbers: ~1-5 seconds
   - Similar cost to encryption

5. KEY OBSERVATIONS:
   - Encryption/Decryption are the bottlenecks
   - Homomorphic operations are relatively fast
   - Paillier is much slower than AES (which is ~microseconds)
   - Trade-off: Security vs. Performance

6. USE CASES:
   - Privacy-preserving computation
   -

## 7. Practical Example: Secure Computation

In [81]:
# Scenario: Computing average salary without revealing individual salaries
print("SCENARIO: Computing Average Salary Securely")
print("="*60)

# Employee salaries (in thousands)
salaries = [50, 60, 75, 80, 95]
print(f"\nEmployee salaries (plaintext): {salaries}")

# Encrypt salaries
encrypted_salaries = [pubkey.encrypt(s) for s in salaries]
print(f"Salaries encrypted (only public key needed)")

# Compute sum on encrypted data
encrypted_sum = encrypted_salaries[0]
for enc_sal in encrypted_salaries[1:]:
    encrypted_sum = encrypted_sum + enc_sal

print(f"Sum computed on encrypted data")

# Decrypt sum (only private key holder can do this)
decrypted_sum = privkey.decrypt(encrypted_sum)
print(f"\nDecrypted sum: {decrypted_sum}")

# Compute average
average = decrypted_sum / len(salaries)
print(f"Average salary: ${average:.2f}k")

# Verify
expected_average = sum(salaries) / len(salaries)
print(f"Expected average: ${expected_average:.2f}k")
print(f"Match: {abs(average - expected_average) < 0.01}")

print(f"\nKey insight: Computation happened on encrypted data!")
print(f"Individual salaries were never revealed to the computer.")

SCENARIO: Computing Average Salary Securely

Employee salaries (plaintext): [50, 60, 75, 80, 95]
Salaries encrypted (only public key needed)
Sum computed on encrypted data

Decrypted sum: 360
Average salary: $72.00k
Expected average: $72.00k
Match: True

Key insight: Computation happened on encrypted data!
Individual salaries were never revealed to the computer.


## 8. Summary: Paillier Cryptosystem

In [82]:
print("="*70)
print("PAILLIER CRYPTOSYSTEM - KEY TAKEAWAYS")
print("="*70)

print("""
1. WHAT IS PAILLIER?
   - Asymmetric (public-key) cryptosystem
   - Partially homomorphic encryption (PHE)
   - Allows computation on encrypted data

2. KEY PROPERTIES:
   - Public key: n = p × q (product of two large primes)
   - Private key: p and q (the prime factors)
   - Security based on difficulty of factoring n

3. HOMOMORPHIC PROPERTIES:
   ✓ Allowed: E(m1) + E(m2) = E(m1 + m2)
   ✓ Allowed: E(m1) × k = E(m1 × k)
   ✗ Not allowed: E(m1) × E(m2) = E(m1 × m2)

4. SECURITY FEATURES:
   - Randomization: Same plaintext → different ciphertexts
   - Large key size: 2048+ bits recommended
   - Semantic security: Ciphertext reveals no information

5. PERFORMANCE CHARACTERISTICS:
   - Key generation: Seconds (expensive)
   - Encryption: Milliseconds per number
   - Homomorphic ops: Milliseconds per operation
   - Decryption: Milliseconds per number
   - Much slower than AES, but enables privacy-preserving computation

6. APPLICATIONS:
   - Secure voting systems
   - Privacy-preserving data analysis
   - Secure multi-party computation
   - Cloud computing with encrypted data
   - Federated learning
   - Secure auctions

7. LIMITATIONS:
   - Only partially homomorphic (can't multiply ciphertexts)
   - Slower than symmetric encryption
   - Ciphertext expansion (large encrypted values)
   - Requires careful key management

8. COMPARISON WITH AES:
   - AES: Fast, symmetric, not homomorphic
   - Paillier: Slow, asymmetric, partially homomorphic
   - Use case: Different security and privacy requirements
""")

print("="*70)

PAILLIER CRYPTOSYSTEM - KEY TAKEAWAYS

1. WHAT IS PAILLIER?
   - Asymmetric (public-key) cryptosystem
   - Partially homomorphic encryption (PHE)
   - Allows computation on encrypted data

2. KEY PROPERTIES:
   - Public key: n = p × q (product of two large primes)
   - Private key: p and q (the prime factors)
   - Security based on difficulty of factoring n

3. HOMOMORPHIC PROPERTIES:
   ✓ Allowed: E(m1) + E(m2) = E(m1 + m2)
   ✓ Allowed: E(m1) × k = E(m1 × k)
   ✗ Not allowed: E(m1) × E(m2) = E(m1 × m2)

4. SECURITY FEATURES:
   - Randomization: Same plaintext → different ciphertexts
   - Large key size: 2048+ bits recommended
   - Semantic security: Ciphertext reveals no information

5. PERFORMANCE CHARACTERISTICS:
   - Key generation: Seconds (expensive)
   - Encryption: Milliseconds per number
   - Homomorphic ops: Milliseconds per operation
   - Decryption: Milliseconds per number
   - Much slower than AES, but enables privacy-preserving computation

6. APPLICATIONS:
   - Secure v